In [561]:
from collections import defaultdict
import numpy as np
import math
#graph data is represented by edges connecting vertices and having a given weight
#vertices is a hashmap: vertices[vertex]=[edge_1,...,edge_n] of all the edges connected to a given vertex
#edges is a hashmap: edges[edge]=(start,end)
#weights is a hashmap: weights[edge]=weight
class graph(object):
    def __init__(self,vertices,edges,weights):
        self.vertices=vertices
        self.edges=edges
        self.weights=weights
        self.embed={}
        self.degrees={}
        self.maxdeg=0
        self.decktransf={}
        self.tau=0
        self.beta=0
    
    def compute_degrees(self):
        for vertex in self.vertices.keys():
            self.degrees[vertex]=len(self.vertices[vertex])
            if self.degrees[vertex]>self.maxdeg:
                self.maxdeg=self.degrees[vertex]
    
    def compute_etamax(self):
        beta=self.beta
        eta=1
        for edge in self.edges.keys():
            start,end=self.edges[edge]
            mustart=2*math.pi/self.degrees[start]-beta
            muend=2*math.pi/self.degrees[end]-beta
            mu=min(mustart,muend)
            L=-2*math.log(math.tan(mu/2))
            eta1=L/self.weights[edge]
            if eta1>1:
                eta=max(eta,eta1)
        return eta
            
    def find_sep(self,epsilon,var=0.000000001): #check that we don't need eta
        self.compute_degrees()
        d=self.maxdeg
        self.beta=math.pi/d
        nu=-2*math.log(math.tan(self.beta/2))
        tau=1
        lowb=nu*(1+epsilon)/epsilon
        etamax=self.compute_etamax()
        for edge in self.edges.keys():
            if tau*self.weights[edge]<lowb:
                tau=max(tau,lowb/self.weights[edge])
        self.tau=max(tau,etamax)
    
    def cayleytransf(self,z):
        x,y=z
        x0=-2*y/((1-x)**2+y**2)
        y0=(1-x**2-y**2)/((1-x)**2+y**2)
        return (x0,y0)
        
    
    def findtransf(self,edge1,edge2): #translate everything on the upper half-plane
        start1,end1=edge1
        start2,end2=edge2
        xs1,ys1=self.cayleytransf(start1) #xs,ys are the coordinates of the starting point of the edge 
        xe1,ye1=self.cayleytransf(end1)   #xe,ye are the coordinates of the endpoint of the edge
        xs2,ys2=self.cayleytransf(start2)
        xe2,ye2=self.cayleytransf(end2)
        if xs1!=xe1: #we distinguish the case in which they lie on a semicircle from when they lie on a vertical axis
            a1=(ys1**2-ye1**2)/(2*(xs1-xe1))+(xs1+xe1)/2 #a semicircle has the form (x-a)**2+y**2=c
            c1=(xs1-a1)**2+ys1**2
            if xs1<xe1:
                rep1=a1-math.sqrt(c1) #for a given semicircle, the edge gives an orientation from start to end
                att1=a1+math.sqrt(c1) #the orientation goes rep->start->end->att. rep and att are the x-coordinate
            else:
                rep1=a1+math.sqrt(c1)
                att1=a1-math.sqrt(c1)
        else: # a line has the form x=a
            a1=xs1
            c1=0
            if ys1<ye1:
                rep1=a1
            else:
                att1=a1
        if xs2!=xe2:
            a2=(ys2**2-ye2**2)/(2*(xs2-xe2))+(xs2+xe2)/2
            c2=(xs2-a2)**2+ys2**2
            if xs2<xe2:
                rep2=a2-math.sqrt(c2)
                att2=a2+math.sqrt(c2)
            else:
                rep2=a2+math.sqrt(c2)
                att2=a2-math.sqrt(c2)
        else:
            a2=xs2
            c2=0
            if ys2<ye2:
                rep2=0
            else:
                att2=0
        if c1!=0:#semicircle case, we find the transformation sending the semicircle to the y axis with the right orientation and the starpoint to i
            base1=((xs1-att1)**2+ys1**2)/((rep1-att1)*ys1)
            mob1=np.array([[base1/math.sqrt(base1*(rep1-att1)),-base1*rep1/math.sqrt(base1*(rep1-att1))],[1/math.sqrt(base1*(rep1-att1)),-att1/math.sqrt(base1*(rep1-att1))]])
        elif ys1<ye1: #vertical line case, we find the transformation sending the line to the y axis and the start point to i
            mob1=np.array([[1/math.sqrt(ys1),-rep1/math.sqrt(ys1)],[0,math.sqrt(ys1)]]) 
        else:
            mob1=np.array([[0,-math.sqrt(ys1)],[1/math.sqrt(ys1),-att1/math.sqrt(ys1)]])
        if c2!=0:
            base2=((xs2-att2)**2+ys2**2)/((rep2-att2)*ys2)
            mob2=np.array([[base2/math.sqrt(base2*(rep2-att2)),-base2*rep2/math.sqrt(base2*(rep2-att2))],[1/math.sqrt(base2*(rep2-att2)),-att2/math.sqrt(base2*(rep2-att2))]])
        elif ys2<ye2:
            mob2=np.array([[1/math.sqrt(ys2),-rep2/math.sqrt(ys2)],[0,math.sqrt(ys2)]])
        else:
            mob2=np.array([[0,-math.sqrt(ys2)],[1/math.sqrt(ys2),-att2/math.sqrt(ys2)]])
        mob2_1=np.linalg.inv(mob2) #find the inverse of the second
        transfplane=mob2_1.dot(mob1) #multiply the first for the inverse of the second
        #print('errors on first vertex:',self.transform(transfplane,(xs1,ys1))[0]-xs2,self.transform(transfplane,(xs1,ys1))[1]-ys2)
        #print('errors on second vertex:',self.transform(transfplane,(xe1,ye1))[0]-xe2,self.transform(transfplane,(xe1,ye1))[1]-ye2)
        cayley=np.array([[1,-1j],[1,1j]])
        cayley_inv=np.array([[-1j,-1j],[1,-1]])
        final=cayley.dot(transfplane.dot(cayley_inv))#send back to the disk model
        #print('errors on first vertex:',self.transform(final,start1)[0]-start2[0],self.transform(final,start1)[1]-start2[1])
        #print('errors on second vertex:',self.transform(final,end1)[0]-end2[0],self.transform(final,end1)[1]-end2[1])

        return final
    
    def transform(self,mob,point):
        z=point[0]+1j*point[1]
        num=mob[0,0]*z+mob[0,1]
        den=mob[1,0]*z+mob[1,1]
        return (np.real(num/den),np.imag(num/den))
    
    def hypdistance(self,a,b):
        a_x,a_y=a
        b_x,b_y=b
        anorm2=a_x**2+a_y**2
        bnorm2=b_x**2+b_y**2
        difnorm2=(a_x-b_x)**2+(a_y-b_y)**2
        delta=(2*difnorm2)/((1-anorm2)*(1-bnorm2))
        dist=np.arccosh(1+delta)
        return dist
        
    
    def calculate_distance(self,point0,point1):
        a=self.embed[point0]
        b=self.embed[point1]
        dist=self.hypdistance(a,b)
        for edge in self.decktransf.keys():
            a1=self.transform(self.decktransf[edge],a)
            a2=self.transform(np.linalg.inv(self.decktransf[edge]),a)
            newdist1=self.hypdistance(a1,b)
            newdist2=self.hypdistance(a2,b)
            if min(newdist1,newdist2)<dist:
                dist=min(newdist1,newdist2)
        return dist/self.tau        
    
    def hypembed(self,start,epsilon): 
        self.find_sep(epsilon)
        beta=self.beta
        tau=self.tau
        print(beta,tau)
        queue=[(start,None)]
        pairings={}
        while queue:
            point,parent=queue[0]
            if parent==None: #during the first iteration, we embed the starting point in (0,0) 
                self.embed[point]=(0,0)
                angle=0 #initialize the angle
                degree=self.degrees[point]
                alpha=2*math.pi/degree-beta 
                i=0
                for edge in self.vertices[point]:
                    r=self.weights[edge]*tau
                    hypr=math.tanh(r/2) #length of a given edge
                    for vertex in self.edges[edge]: #if the endpoint hasn't been embedded, embed it
                        if vertex!=point:
                            if vertex not in self.embed.keys():
                                self.embed[vertex]=(hypr*math.cos(angle),hypr*math.sin(angle)) 
                                queue.append((vertex,edge)) #save the parent edge
                            elif edge not in pairings.keys(): #if the endpoint has been embedded, but the edge hasn't, save the edge
                                pairings[edge]=((0,0),(hypr*math.cos(angle),hypr*math.sin(angle)))
                            elif edge not in self.decktransf.keys(): #if the edge has been embedded, find the transformation
                                self.decktransf[edge]=self.findtransf(pairings[edge],((0,0),(hypr*math.cos(angle),hypr*math.sin(angle))))
                    i+=1
                    angle=((2*i)/degree)*math.pi
            else: #now we have saved the parent edge
                point1,point2=self.edges[parent]
                degree=self.degrees[point]
                alpha=2*math.pi/degree-2*beta
                if point1!=point:
                    parentpoint=point1
                else:
                    parentpoint=point2
                r0=self.weights[parent]*tau
                hypr0=math.tanh(r0/2)
                mob=self.findtransf((self.embed[parentpoint],self.embed[point]),((hypr0,0),(0,0)))
                mob_inv=np.linalg.inv(mob)
                angle=alpha+beta
                i=1
                angle=((2*i)/degree)*math.pi
                for edge in self.vertices[point]:
                    if edge!=parent:
                        r=self.weights[edge]*tau
                        hypr=math.tanh(r/2)
                        for vertex in self.edges[edge]:
                            if vertex!=point:
                                if vertex not in self.embed.keys():
                                    self.embed[vertex]=self.transform(mob_inv,(hypr*math.cos(angle),hypr*math.sin(angle)))
                                    queue.append((vertex,edge))
                                elif edge not in pairings.keys():
                                    newcoord=self.transform(mob_inv,(hypr*math.cos(angle),hypr*math.sin(angle)))
                                    pairings[edge]=(self.embed[point],newcoord)
                                elif edge not in self.decktransf.keys():
                                    newcoord=self.transform(mob_inv,(hypr*math.cos(angle),hypr*math.sin(angle)))
                                    self.decktransf[edge]=self.findtransf(pairings[edge],(newcoord,self.embed[point]))
                        i+=1
                        angle=((2*i)/degree)*math.pi
            del queue[0]
            
                        
                
        
            
        

In [562]:
vertices={'A':['AB','AC'],'B':['AB','BC'],'C':['AC','BC']}
edges={'AB':('A','B'),'AC':('A','C'),'BC':('B','C')}
weights={'AB':1,'AC':1,'BC':1}

In [563]:
trigraph=graph(vertices,edges,weights)

In [564]:
trigraph.compute_degrees()
print(trigraph.degrees)

{'A': 2, 'B': 2, 'C': 2}


In [565]:
trigraph.hypembed('A',1)

1.5707963267948966 1
1 0.46211715726000974
1 0.46211715726000974


In [566]:
print(trigraph.embed,trigraph.decktransf)

{'A': (0, 0), 'B': (0.46211715726000974, 0.0), 'C': (-0.46211715726000974, 5.65930297469545e-17)} {'BC': array([[-1.66909298e-16+4.70481923j, -1.66909298e-16-4.25855891j],
       [ 1.66909298e-16-4.25855891j,  1.66909298e-16+4.70481923j]])}


In [567]:
print(trigraph.calculate_distance('B','C'))

0.9999999999999997


In [568]:
k33vertices={'1':['12','13','16'],'2':['12','24','25'],'3':['13','34','35'],'4':['24','34','46'],'5':['35','25','56'],'6':['16','46','56']}
k33edges={'12':('1','2'),'13':('1','3'),'16':('1','6'),'24':('2','4'),'25':('2','5'),'34':('3','4'),'35':('3','5'),'46':('4','6'),'56':('5','6')}
k33weights={'12':1,'13':1,'16':1,'24':1,'25':1,'34':1,'35':1,'46':1,'56':1}

In [569]:
k33graph=graph(k33vertices,k33edges,k33weights)

In [590]:
k33graph.hypembed('1',0.1)

1.0471975511965976 12.084735175349213
12.084735175349213 0.9999887100051934
12.084735175349213 0.9999887100051934
12.084735175349213 0.9999887100051934


In [594]:
print(k33graph.embed,k33graph.decktransf)

{'1': (0, 0), '2': (0.8000000000000002, 0.0), '3': (-0.3999999999999999, 0.6928203230275511), '6': (-0.4000000000000004, -0.6928203230275508), '4': (0.9601873536299768, -0.12168975228821136), '5': (0.9601873536299769, 0.1216897522882113)} {'34': array([[-25.66001196+46.44444444j,  32.07501495+42.22222222j],
       [-32.07501495+42.22222222j,  25.66001196+46.44444444j]]), '46': array([[-25.66001196-46.44444444j,  20.52800957-48.88888889j],
       [-20.52800957-48.88888889j,  25.66001196-46.44444444j]]), '35': array([[ 25.66001196-46.44444444j, -20.52800957-48.88888889j],
       [ 20.52800957-48.88888889j, -25.66001196-46.44444444j]]), '56': array([[ 25.66001196+46.44444444j, -32.07501495+42.22222222j],
       [ 32.07501495+42.22222222j, -25.66001196+46.44444444j]])}


In [595]:
print(k33graph.calculate_distance('1','2'))

0.18181818181818177
